In [2]:
PAGE = "https://quizack.com/django/questions-and-answers"
PAGE2 = "https://quizack.com/django/questions-and-answers/2"

In [3]:
import requests

page_get_response = requests.get(url=PAGE)
page2_get_response = requests.get(url=PAGE2)

In [10]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page_get_response.text, 'html.parser')
soup2 = BeautifulSoup(page_get_response.text, 'html.parser')

In [73]:
from collections import namedtuple
Question = namedtuple("Question", ("number", "link", "text"))

def add_questions(soup_object):
    for block in soup_object.find_all(attrs={"class": "col-lg-6 col-md-6 col-sm-12 mt-2"}):
        question_html = block.contents[1].contents[1].contents[1].contents
        q_numb = question_html[1].text
        q_link = question_html[3].attrs["href"]
        q_text = question_html[3].contents[0].text
        
        q = Question(q_numb, q_link, q_text)
        print(q)
        
        break

In [74]:
questions = []

add_questions(soup)




Question(number='1.', link='https://quizack.com/django/mcq/which-is-fictional-http-request-method', text='Which is Fictional HTTP request method?')
